In [154]:
import pandas as pd
from pybaseball import statcast
from pybaseball import schedule_and_record
from datetime import datetime
import time
pd.options.display.max_rows=100
pd.options.display.max_columns=100

In [46]:
df1=pd.read_csv("abs_5-11.csv")
df2=pd.read_csv("abs_6-24.csv")

In [60]:
df1.shape,df2.shape

((38831, 89), (40000, 89))

In [61]:
df=pd.concat([df1,df2],axis=0).reset_index(drop=True)

In [62]:
df.player_name.nunique()

777

In [63]:
df.events.value_counts()

field_out                    34974
strikeout                    19678
single                       12281
double                        3980
home_run                      2635
force_out                     1750
grounded_into_double_play     1682
sac_fly                        564
triple                         401
sac_bunt                       383
double_play                    213
fielders_choice_out            149
strikeout_double_play           86
fielders_choice                 47
sac_fly_double_play              6
triple_play                      1
sac_bunt_double_play             1
Name: events, dtype: int64

In [64]:
df.events.isnull().sum()

0

In [65]:
df['hit']=df.events.apply(lambda x: 1 if x in ["single",'double','home_run','triple'] else 0)
df['ab']=df.events.apply(lambda x: 0 if x in ["sac_fly",'sac_bunt','sac_fly_double_play','sac_bunt_double_play'] else 1)

In [66]:
df['date']=pd.to_datetime(df['game_date'],infer_datetime_format=True)
sorted(df.date.unique())

[numpy.datetime64('2018-03-29T00:00:00.000000000'),
 numpy.datetime64('2018-03-30T00:00:00.000000000'),
 numpy.datetime64('2018-03-31T00:00:00.000000000'),
 numpy.datetime64('2018-04-01T00:00:00.000000000'),
 numpy.datetime64('2018-04-02T00:00:00.000000000'),
 numpy.datetime64('2018-04-03T00:00:00.000000000'),
 numpy.datetime64('2018-04-04T00:00:00.000000000'),
 numpy.datetime64('2018-04-05T00:00:00.000000000'),
 numpy.datetime64('2018-04-06T00:00:00.000000000'),
 numpy.datetime64('2018-04-07T00:00:00.000000000'),
 numpy.datetime64('2018-04-08T00:00:00.000000000'),
 numpy.datetime64('2018-04-09T00:00:00.000000000'),
 numpy.datetime64('2018-04-10T00:00:00.000000000'),
 numpy.datetime64('2018-04-11T00:00:00.000000000'),
 numpy.datetime64('2018-04-12T00:00:00.000000000'),
 numpy.datetime64('2018-04-13T00:00:00.000000000'),
 numpy.datetime64('2018-04-14T00:00:00.000000000'),
 numpy.datetime64('2018-04-15T00:00:00.000000000'),
 numpy.datetime64('2018-04-16T00:00:00.000000000'),
 numpy.datet

In [67]:
df['htd']=df.sort_values(['date','inning'],ascending=True).groupby(['player_name']).hit.cumsum()
df['abtd']=df.sort_values(['date','inning'],ascending=True).groupby(['player_name']).ab.cumsum()
df['avg']=df.htd/df.abtd

In [68]:
df[['player_name','date','inning','hit','ab','htd','abtd','avg']].sort_values(['player_name','date','inning'],ascending=True).head(100)

,player_name,date,inning,hit,ab,htd,abtd,avg
34342,A.J. Cole,2018-04-03,2,1,1,1,1,1.000000
34191,A.J. Cole,2018-04-03,3,0,1,1,2,0.500000
27480,A.J. Cole,2018-04-11,3,0,0,1,2,0.500000
27261,A.J. Cole,2018-04-11,5,0,1,1,3,0.333333
35206,A.J. Ellis,2018-04-02,2,0,1,0,1,0.000000
34810,A.J. Ellis,2018-04-02,6,0,1,0,2,0.000000
34643,A.J. Ellis,2018-04-02,8,0,1,0,3,0.000000
34333,A.J. Ellis,2018-04-03,2,0,1,0,4,0.000000
34102,A.J. Ellis,2018-04-03,5,1,1,1,5,0.200000
33956,A.J. Ellis,2018-04-03,6,0,1,1,6,0.166667


In [69]:
df['player_team']=df.apply(lambda x: x.home_team if x.inning_topbot=="Bot" else x.away_team,axis=1)

In [70]:
teams=df.player_team.unique()

In [155]:
team_schedule={}
missed=[]
for t in teams:
    try:
        d=schedule_and_record(2018,t)
        d['fix_date']=d.Date + " 2018"
        d['game_date']=pd.to_datetime(d.fix_date.apply(lambda x: datetime.strptime(x,"%A, %b %d %Y")).apply(lambda x: x.strftime("%Y-%m-%d")),infer_datetime_format=True)
        d['place']=d.Home_Away.apply(lambda x: "Home" if x=="Home" else "Away")
        d2=d[d.game_date>=datetime.today()][['Tm','place',"Opp","game_date"]]
        team_schedule[t]=d2
        time.sleep(3)
    except ValueError:
        missed.append(t)

In [149]:
schedule_and_record(2018,'CLE')

,Date,Tm,Home_Away,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,Streak,Orig. Scheduled
1,"Thursday, Mar 29",CLE,@,SEA,L,1.0,2.0,9.0,0-1,3.0,1.0,Hernandez,Kluber,Diaz,2:35,N,47149.0,-1.0,None
2,"Saturday, Mar 31",CLE,@,SEA,W,6.0,5.0,9.0,1-1,2.0,1.0,Carrasco,Paxton,Allen,2:47,D,35881.0,1.0,None
3,"Sunday, Apr 1",CLE,@,SEA,L,4.0,5.0,9.0,1-2,3.0,1.5,Leake,Otero,Diaz,2:57,D,24506.0,-1.0,None
4,"Monday, Apr 2",CLE,@,LAA,W,6.0,0.0,9.0,2-2,2.0,0.5,Clevinger,Ramirez,None,3:25,N,43904.0,1.0,None
5,"Tuesday, Apr 3",CLE,@,LAA,L,2.0,13.0,9.0,2-3,3.0,0.5,Richards,Tomlin,None,2:49,N,35007.0,-1.0,None
6,"Wednesday, Apr 4",CLE,@,LAA,L-wo,2.0,3.0,13.0,2-4,3.0,1.5,Ramirez,McAllister,None,4:28,D,32412.0,-2.0,None
7,"Friday, Apr 6",CLE,Home,KCR,W,3.0,2.0,9.0,3-4,3.0,1.5,Carrasco,Duffy,Allen,2:41,D,34720.0,1.0,None
8,"Saturday, Apr 7",CLE,Home,KCR,L,0.0,1.0,9.0,3-5,4.0,1.5,Kennedy,Bauer,Herrera,2:29,D,17362.0,-1.0,None
9,"Sunday, Apr 8",CLE,Home,KCR,W-wo,3.0,1.0,9.0,4-5,3.0,1.0,Allen,Maurer,None,2:55,D,14240.0,1.0,None
10,"Monday, Apr 9",CLE,Home,DET,W,2.0,0.0,9.0,5-5,1.0,Tied,Kluber,Liriano,Miller,2:39,N,9843.0,2.0,None


In [156]:
missing=[team_schedule[x].Opp.unique() for x in team_schedule.keys()]
missed

['CWS',
 'CLE',
 'TOR',
 'CHC',
 'KC',
 'SF',
 'PIT',
 'LAD',
 'MIN',
 'NYY',
 'CIN',
 'OAK',
 'ATL',
 'NYM',
 'TB',
 'BAL',
 'WSH',
 'SEA',
 'DET']

In [147]:
set([y for x in missing for y in x])
team_dict={"CWS":"CHW",'CLE':'CLE'}

{u'ARI',
 u'ATL',
 u'BAL',
 u'BOS',
 u'CHC',
 u'CHW',
 u'CIN',
 u'CLE',
 u'COL',
 u'DET',
 u'HOU',
 u'KCR',
 u'LAA',
 u'LAD',
 u'MIA',
 u'MIL',
 u'MIN',
 u'NYM',
 u'NYY',
 u'OAK',
 u'PHI',
 u'PIT',
 u'SDP',
 u'SEA',
 u'SFG',
 u'STL',
 u'TBR',
 u'TEX',
 u'TOR',
 u'WSN'}

In [127]:
team_schedule['MIL']

,Tm,place,Opp,game_date
78,MIL,Home,KCR,2018-06-26
79,MIL,Home,KCR,2018-06-27
80,MIL,Away,CIN,2018-06-28
81,MIL,Away,CIN,2018-06-29
82,MIL,Away,CIN,2018-06-30
83,MIL,Away,CIN,2018-07-01
84,MIL,Home,MIN,2018-07-02
85,MIL,Home,MIN,2018-07-03
86,MIL,Home,MIN,2018-07-04
87,MIL,Home,ATL,2018-07-05


In [103]:
a=d.Date[1] + " 2018"
g=datetime.strptime(a,"%A, %b %d %Y")
print(g.strftime("%Y-%m-%d"))

2018-03-29


In [109]:
d['fix_date']=d.Date + " 2018"
d['game_date']=pd.to_datetime(d.fix_date.apply(lambda x: datetime.strptime(x,"%A, %b %d %Y")).apply(lambda x: x.strftime("%Y-%m-%d")),infer_datetime_format=True)

In [110]:
d.game_date

1     2018-03-29
2     2018-03-30
3     2018-03-31
4     2018-04-02
5     2018-04-03
6     2018-04-04
7     2018-04-05
8     2018-04-06
9     2018-04-07
10    2018-04-08
11    2018-04-09
12    2018-04-10
13    2018-04-11
14    2018-04-12
15    2018-04-13
16    2018-04-14
17    2018-04-15
18    2018-04-16
19    2018-04-17
20    2018-04-18
21    2018-04-20
22    2018-04-21
23    2018-04-22
24    2018-04-23
25    2018-04-24
26    2018-04-25
27    2018-04-27
28    2018-04-28
29    2018-04-29
30    2018-04-30
31    2018-05-01
32    2018-05-02
33    2018-05-04
34    2018-05-05
35    2018-05-06
36    2018-05-08
37    2018-05-09
38    2018-05-10
39    2018-05-11
40    2018-05-12
41    2018-05-13
42    2018-05-14
43    2018-05-15
44    2018-05-17
45    2018-05-18
46    2018-05-19
47    2018-05-20
48    2018-05-21
49    2018-05-22
50    2018-05-23
         ...    
113   2018-08-07
114   2018-08-08
115   2018-08-09
116   2018-08-10
117   2018-08-11
118   2018-08-12
119   2018-08-14
120   2018-08-

In [73]:
d['game_date']=pd.to_datetime(d['Date'],infer_datetime_format=True)
d

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1-03-29 00:00:00

In [59]:
df.to_csv("all_abs_edit.csv")

In [15]:
## statcast test
%timeit
data=statcast()

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [3]:
data.shape

(3815, 90)

In [7]:
data[data.events.notnull()].shape

(976, 90)